# <center> **Deep Learning: Supervised Seafloor classification with CNN** </center> 
## <center> Machine Learning Programming Exercise 8 part 2.1: **From scratch**</center>

| <font size=6,font color='red'>Monôme / binôme</font> | <font size=6,font color='red'>Nom</font> | <font size=6,font color='red'>Prénom</font> |
|:-------------: |:----------- |:------ |
| binôme 1 | <span style="color:red">DUBEE</span> | <span style="color:red">Melvin</span> |
| binôme 2 | <span style="color:red">ROUDAUT</span> | <span style="color:red">Tanguy</span> |


Vous proposerez une architecture de réseau profond convolutif, apprendrez le modèle avec les patchs et évaluerez ses performances. 
- Expliquez votre architecture et en particulier à quoi servent les couches (et leur enchainement) de votre architecture.
- Vous comparerez ensuite les performances obtenus (par rapport à celles obtenues à la partie précédente) sur la matrice de confusion et les métriques de performance classiques.

- Suivre les différentes étapes du tuto.
- **remarque**: comme les images sonar sont en niveaux de gris, il s'agira de conserver un seul canal.

- Enfin, vous évaluerez proprement les performances obtenues (learning curves, matrice de confusion).

# 1. Import useful packages 
Pour pouvoir commencer, vous importerez les librairies suivantes:

## 1.1 Colab or not colab

In [ ]:
# common imports
import sys,os,glob

# Colab preamble
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:

  # mount google drive directories
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True) 


  # ----------- Your code here --------------------->
  # replace the ipynb_name (below) with the name of your jupyter notebook file

  ipynb_name = 'tp2.2_learning_cnns_from_scratch_startecode.ipynb'

  # ------------------------------------------------>

  ipynb_name = glob.glob(os.getcwd() + '/**/' + ipynb_name, recursive = True)
  code_folder = os.path.dirname(ipynb_name[0])

  # change to the right folder
  %cd "$code_folder"
  !ls


## 1.2 Import packages

In [ ]:
# Common imports
import os
from time import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from functools import partial
import pickle

# machine learning packages
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img

print(tf.config.list_physical_devices())
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IN_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

       

## 1.3 Useful codes

In [ ]:
 
# Définition du chemin vers le répertoire dataset pour les images et les labels
DATASET_PATH = r'./dataset/imgs/'
LABEL_PATH = r'./dataset/labels/labels.csv'

# Flag pour le chargement des images
# True for fine tuning | False for from scratch
flag_load_as_rgb = False 

# Taille d'entrée du modèle (>=71 pour xception)
target_size = 200

# Import des données
def importData():

    # Charger le fichier CSV
    dataset_df = pd.read_csv(LABEL_PATH)

    # We add another column to the labels dataset to identify image path
    dataset_df['image_path'] = dataset_df.apply(lambda row: (DATASET_PATH + row["id"]), axis=1)

    # Récupération des labels
    label_names = dataset_df['seafloor']

    # Chargement des images


    # Changement de la taille des images et duplication sur canaux RGB
   
    batch_imgs = []
    for img in dataset_df['image_path'].values.tolist():
        
        if flag_load_as_rgb:
            tmp = load_img(img, color_mode = "rgb", target_size=(target_size, target_size))
        else:
            tmp = load_img(img, color_mode = "grayscale", target_size=(target_size, target_size))
        
        # Converts a PIL Image instance to a Numpy array
        tmp = img_to_array(tmp)
        batch_imgs.append(tmp)
    
    # conversion en numpy array
    batch_imgs = np.array(batch_imgs).astype('float32')
            
    return batch_imgs, label_names

# call importData
batch_imgs, label_names = importData()

# variables utiles
instance_nb, height, width, channel_nb = batch_imgs.shape
feature_nb = batch_imgs.shape[1]*batch_imgs.shape[2]
channel_nb = batch_imgs.shape[-1]

print('dimension du batch d''images: {}'.format(batch_imgs.shape))
print('dimension des labels: {}'.format(label_names.shape))

# ---------------------------------------------------------------------------------------------------------------
# PREPARE DATASETS Split into 3 sets
# ---------------------------------------------------------------------------------------------------------------
def prepare_datasets(batch_imgs, label_names):
    print('Split into 3 sets...')

    tmp = train_test_split(batch_imgs,
                           label_names,
                           test_size=0.5,
                           stratify=np.array(label_names),
                           random_state=42)
    batch_imgs_train, batch_imgs_test, labelNames_train, labelNames_test = tmp
    
    tmp = train_test_split(batch_imgs_test,
                           labelNames_test,
                           test_size=0.5,
                           stratify=np.array(labelNames_test),
                           random_state=42)
    batch_imgs_test, batch_imgs_val, labelNames_test, labelNames_val = tmp
    
    # taille du dataset
    dataset_size = batch_imgs.shape[0]

    # nb de classes
    labelNames_unique = label_names.unique()
    label_nb = labelNames_unique.shape[0]


    return batch_imgs_train, labelNames_train, batch_imgs_val, labelNames_val, batch_imgs_test, labelNames_test

# call prepare_datasets
batch_imgs_train, labelNames_train, batch_imgs_val, labelNames_val, batch_imgs_test, labelNames_test = prepare_datasets(batch_imgs, label_names)

# Vérification des formats des ensembles
print("Format du set de train : "     , batch_imgs_train.shape)
print("Format du set de validation : ", batch_imgs_val.shape)
print("Format du set de test : "      , batch_imgs_test.shape)


# ---------------------------------------------------------------------------------------------------------------
#  transformation des labels selon différents codages
# ---------------------------------------------------------------------------------------------------------------

#  Noms des labels
labelNames_unique = label_names.unique()

# nb de classes
label_nb = labelNames_unique.shape[0]

# enc labelNames to indices
encName2Ind = preprocessing.LabelEncoder()
encName2Ind.fit(labelNames_unique)
labelIndices_unique = encName2Ind.transform(labelNames_unique)
labelIndices  = encName2Ind.transform(label_names)

# enc indices to  one-hot-encoding
encInd2Ohe = preprocessing.OneHotEncoder(sparse=False)
encInd2Ohe.fit(labelIndices_unique.reshape(-1, 1))
labelOhe = encInd2Ohe.transform(labelIndices.reshape(-1, 1))

# enc labelNames to  one-hot-encoding
encName2Ohe = preprocessing.OneHotEncoder(sparse=False)
encName2Ohe.fit(labelNames_unique.reshape(-1, 1))
#labelOhe2 = encName2Ohe.transform(label_names.reset_index(drop=True).values.reshape(-1, 1))


# Conversion des noms des labels en indices
labelInd_train = encName2Ind.transform(labelNames_train)
labelInd_val = encName2Ind.transform(labelNames_val)
labelInd_test = encName2Ind.transform(labelNames_test)

# Conversion des noms des labels en  one-hot-encoding
labelOhe_train = encName2Ohe.transform(labelNames_train.reset_index(drop=True).values.reshape(-1, 1))
labelOhe_val   = encName2Ohe.transform(labelNames_val.reset_index(drop=True).values.reshape(-1, 1))
labelOhe_test  = encName2Ohe.transform(labelNames_test.reset_index(drop=True).values.reshape(-1, 1))

# autre solution avec panda
# labelOhe_train = pd.get_dummies(labelNames_train.reset_index(drop=True)).values
# labelOhe_val   = pd.get_dummies(labelNames_val.reset_index(drop=True)).values
# labelOhe_test  = pd.get_dummies(labelNames_test.reset_index(drop=True)).values



# ---------------------------------------------------------------------------------------------------------------
# DATASETS Summary
# ---------------------------------------------------------------------------------------------------------------
#  Noms et indices des labels
labelNames_unique   = label_names.unique()
labelIndices_unique = encName2Ind.transform(labelNames_unique)
label_nb            = labelNames_unique.shape[0]

# taille du dataset
dataset_size = batch_imgs.shape[0]

print('------------------------------')
print('Seafloor Training Set Summary ')
print('------------------------------')
print('Feature Shape:', batch_imgs_train.shape)
print('Labels Shape:', labelNames_train.shape)
print('labels distrib over labels:', [sum(labelInd_train == ind) for ind in labelIndices_unique])
print('------------------------------')
print('Seafloor Validation Set Summary ')
print('------------------------------')
print('Validation Features Shape:', batch_imgs_val.shape)
print('Validation Labels Shape:', labelNames_val.shape)
print('labels distrib over classe:', [sum(labelInd_val == ind) for ind in labelIndices_unique])
print('------------------------------')
print('Seafloor Testing Set Summary ')
print('------------------------------')
print('Testing Features Shape:', batch_imgs_test.shape)
print('Testing Labels Shape:', labelNames_test.shape)
print('labels distrib over classe:', [sum(labelInd_test == ind) for ind in labelIndices_unique])
print('------------------------------')
print('Split into 3 sets...done')


print('Encoding done...')


# ---------------------------------------------------------------------------------------------------------------
# Normalisation 
# ---------------------------------------------------------------------------------------------------------------
# estimation
images_mean = batch_imgs_train.mean(axis=0, keepdims=True)
images_std = batch_imgs_train.std(axis=0, keepdims=True) + 1e-7

# normalisation
batch_imgs_train = (batch_imgs_train - images_mean) / images_std
batch_imgs_val = (batch_imgs_val - images_mean) / images_std
batch_imgs_test = (batch_imgs_test - images_mean) / images_std



# ---------------------------------------------------------------------------------------------------------------
# Création du dataset en objet tf.data.Dataset
# ---------------------------------------------------------------------------------------------------------------

ds_train = tf.data.Dataset.from_tensor_slices((batch_imgs_train, labelInd_train))
ds_val = tf.data.Dataset.from_tensor_slices((batch_imgs_val, labelInd_val))
ds_test = tf.data.Dataset.from_tensor_slices((batch_imgs_test, labelInd_test))

# variables utiles
train_instance_nb = ds_train.cardinality().numpy()
val_instance_nb = ds_val.cardinality().numpy()
test_instance_nb = ds_test.cardinality().numpy()



### 5.1 Paramètres (optionnel)

In [ ]:
# ----------- Your code here --------------------->



# ------------------------------------------------>

### 5.2 Data preprocessing (normalisation, data augmentation)

In [ ]:
# ----------- Your code here --------------------->


# ------------------------------------------------>

### 5.3 Model definition

In [ ]:
#----------- Your code here --------------------->




#------------------------------------------------>
model.summary()


### 5.4 Learning the model 

In [ ]:
#----------- Your code here --------------------->

#------------------------------------------------>

### 5.5 Evaluating the model 

In [ ]:
#----------- Your code here --------------------->



#------------------------------------------------>

### 4.6 Saving the model 

In [ ]:
# Save model

# description
json_finename = RESULT_PATH+'/'+MODEL_TYPE+"_fromscratchmodel.json"
model_finename = RESULT_PATH+'/'+MODEL_TYPE+"_fromscratchmodel.h5"
weights_filename = RESULT_PATH+'/'+MODEL_TYPE+"_fromscratchweights.h5"
hist_filename = RESULT_PATH+'/'+MODEL_TYPE+"_fromscratchhistory.npy"


# save yaml model
model_json = model.to_json()
with open(json_finename, "w") as json_file:
     json_file.write(model_json)

# save model and weights
model.save(model_finename)

# save weights of the model
model.save_weights(weights_filename)
print("Model saved to disk")

# Fit history saving
np.save(hist_filename, history.history)



**Question 5.1: Expliquez votre architecture et en particulier à quoi servent les couches (et leur enchainement) de votre architecture.**

_Double-cliquez ici pour écrire votre réponse_

**Question 5.2: Faites un bilan des performances obtenus.**

_Double-cliquez ici pour écrire votre réponse_

## 6. Bilan des modèles appris


**Question 6.1: Comparez les performances obtenues (par rapport à celles obtenues à la partie précédente) sur la matrice de confusion et les métriques de performance classiques. Et donnez les avantages et inconvénients de chaque approche.**

_Double-cliquez ici pour écrire votre réponse_

## 7. Fonctions d'aide éventuelle


In [ ]:
# fonction d'aide pour afficher les courbes d'apprentissage

def learningCurves(history,title):
    #Learning curve plotting
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    t = f.suptitle(title, fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    epochs = list(range(1,NB_EPOCHS+1))
    ax1.plot(epochs, history['accuracy'], label='Train Accuracy')
    ax1.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    ax1.set_xticks(epochs)
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    ax1.legend()

    ax2.plot(epochs, history['loss'], label='Train Loss')
    ax2.plot(epochs, history['val_loss'], label='Validation Loss')
    ax2.set_xticks(epochs)
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    ax2.legend()

    
# fonction callback (à rajouter en option à model.fit) pour suivre l'évolution de la matrice de confusion au long de l'apprentissage
# credit: K. Bedin (ROB 2020) et D. Eleye (ROB 2023) développé lors du cours
class ConfusionEvaluation(keras.callbacks.Callback):
    """
        Fonction callback pour la méthode 'fit_generator()' permettant d'afficher 
        la matrice de confusion à chaque fin d'Epoch.
        Cela permet de visualiser concraitement l'évolution de la classification au cours de l'apprentissage.
    """
    def __init__(self, validation_data=()):
        super(keras.callbacks.Callback, self).__init__()
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        preds_Inception = self.model.predict(self.X_val)
        matrixInception = confusion_matrix(self.y_val,preds_Inception.argmax(axis=1))
        print("\nConfusion Matrix:")
        print(matrixInception)

# cbk_matconf = ConfusionEvaluation(validation_data=(ds_val, labelInd_val))



    